# core

> Fill in a module description here

In [51]:
#| default_exp core

In [52]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [53]:
#| export
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dataclasses import dataclass
import typing

In [54]:
#| hide
sns.set_style("white")
sns.set_style("ticks")

Create an instance of the class CalibrationModel by passing in the path to both your calibration data and your sample data. Next specify the name of your response variable and the nummber of test replicates you measured.

In [55]:
#| export
class CalibrationModel:

    def __init__(self, x, y, test_replicates):
      
        self.x = x
        self.y = y
        self.slope = None
        self.intercept = None
        self.r_squared = None
        self.std_err = None
        self.test_replicates = test_replicates
        self.cal_line_points = self.x.shape[0]
        self.df_resid = self.cal_line_points - 2
        self.sr = None
        self.mean_replicate_observations = None
        self.average_response_cal_line = None

    def fit_ols(self):

        self.slope, self.intercept, self.r_squared, self.std_err, _ = sp.stats.linregress(self.x, self.y)
    
    def calculate_fitted_values(self):
        
        self.fitted_values = self.slope * self.x + self.intercept

    
    def calculate_sse(self):
        self.calculate_fitted_values()
        return np.sum((self.fitted_values - self.y) **2)
    
    def calculate_syx(self):
        return np.sqrt((self.calculate_sse())/(len(self.x)-2))

    def get_t_value(self,alpha):
        return sp.stats.t.ppf(1 - alpha/2, self.df_resid)

    def calculate_uncertainty(self):
        return self.calculate_sxhat() * self.get_t_value(0.05)
    

    
    def calculate_hibbert_uncertainty(self, sr, test_repeats, y0, average_response_cal_line, sumsquares):
        return (1/self.slope) * np.sqrt(((sr**2)/test_repeats) + ((self.calculate_syx()**2)/self.cal_line_points) + 
                                                                   (((self.calculate_syx()**2)*((y0-average_response_cal_line)**2))/((self.slope**2)*(sumsquares))))

    
    def calculate_sxhat(self):
        return (self.calculate_syx() / self.slope) * np.sqrt( 1/ self.test_replicates + 1 / self.cal_line_points) 
    
    def fit_model(self):
        self.fit_ols()
        self.calculate_uncertainty()
        self.tabulate_results()

    def inverse_prediction(self, unknown):

        if len(unknown) > 1:
            y0 = np.mean(unknown)
            sr = np.std(unknown)
            test_repeats = len(unknown)
            average_response_cal_line = np.mean(self.y)
            sumsquares = np.sum((self.x - self.x.mean())**2)

            pred =  (y0 - self.intercept)/self.slope
        
            return f"{pred} ± {self.calculate_hibbert_uncertainty(sr=sr, y0=y0, test_repeats=test_repeats, average_response_cal_line=average_response_cal_line, sumsquares=sumsquares) * self.get_t_value(0.05)}"
        else:
            y = unknown[0]
            pred = (y - self.intercept)/self.slope
            return f"{pred} ± {self.calculate_uncertainty()}"

    def tabulate_results(self):
        print(f"Calibration curve")
        print(f"R2 = {self.r_squared}")
        print(f"Slope = {self.slope}")
        print(f"Intercept = {self.intercept}")
        print(f"Uncertainity = {self.calculate_uncertainty()}")
        # print(f"Prediction = {self.inverse_prediction()}")

## Tests

In [56]:

def generate_test_data(slope, intercept):
        x = np.linspace(1, 10, num=5)
        y = intercept + x * slope
        df = pd.DataFrame({'concentration': x, "abs": y})
        return df
def generate_sample_data():
    x = np.array(['unknown1', 'unknown2'])
    y = np.array([13.75, 20.50])
    df = pd.DataFrame({'sample': x, "abs": y})
    df = df.set_index('sample')
    return df


# test_data = generate_test_data(3, 4)
# sample_data = generate_sample_data()

test_data = pd.DataFrame({'concentration': [0.2, 0.05, 0.1, 0.8, 0.6, 0.4], "abs": [0.221, 0.057, 0.119, 0.73, 0.599, 0.383]})
sample_data = pd.DataFrame({'unknown': [0.490, 0.471, 0.484, 0.473, 0.479, 0.492]})

In [57]:

cal = CalibrationModel(x=test_data['concentration'], y=test_data['abs'], test_replicates=6)
cal.fit_ols()
print(cal.calculate_uncertainty())
cal.inverse_prediction(sample_data['unknown'])
# print(cal.slope)
# print(cal.intercept)
# print(cal.r_squared)
# print(cal.calculate_syx())

0.036768287028466705


'0.5020733029033536 ± 0.031053583676141718'

In [58]:
cal.fit_model()

Calibration curve
R2 = 0.9976282521058687
Slope = 0.9044109330819979
Intercept = 0.027419415645617395
Uncertainity = 0.036768287028466705


In [59]:
cal.inverse_prediction(sample_data['unknown'])

'0.5020733029033536 ± 0.031053583676141718'

In [60]:
cal.tabulate_results()

Calibration curve
R2 = 0.9976282521058687
Slope = 0.9044109330819979
Intercept = 0.027419415645617395
Uncertainity = 0.036768287028466705


In [61]:
#| hide
import nbdev; nbdev.nbdev_export()